<a href="https://colab.research.google.com/github/fulltrick/-/blob/main/stable-diffusion-forge-py310.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================#
#  Stable Diffusion WebUI Forge版 を Python 3.10 で起動する版
#  - Colab カーネルは 3.11/3.12 でもOK（実行系だけ 3.10）
#  - micromamba で Python 3.10.13 環境を作成
#  - WAI-NSFW-illustrious-SDXL モデルを使用
#  - Google Driveと連携してモデルを保存
#  - --share オプションで外部アクセス可能
# ==========================================================

import os
import sys
import json
import shutil
import subprocess
import threading
import time
import urllib.error
import urllib.request
import importlib.util
from pathlib import Path

# IPython / numpy のインポート（オプション）
IPY_SPEC = importlib.util.find_spec("IPython")
DISPLAY_SPEC = importlib.util.find_spec("IPython.display") if IPY_SPEC else None

if DISPLAY_SPEC is not None:
    from IPython.display import Audio, display
else:
    class _DummyAudio:
        def __init__(self, *, data, rate, autoplay):
            self.data = data
            self.rate = rate
            self.autoplay = autoplay
        def __repr__(self):
            duration = len(self.data) / self.rate if self.rate else 0
            return f"Audio(dry-run, {duration:.2f}s, autoplay={self.autoplay})"
    def display(obj):
        print(f"🔈 {obj}")
    def Audio(*, data, rate, autoplay):
        return _DummyAudio(data=data, rate=rate, autoplay=autoplay)

NP_SPEC = importlib.util.find_spec("numpy")
if NP_SPEC is not None:
    import numpy as np
else:
    np = None

# Google Colab のインポート
GOOGLE_SPEC = importlib.util.find_spec("google")
colab_spec = importlib.util.find_spec("google.colab") if GOOGLE_SPEC else None

if colab_spec is None:
    class _DummyDrive:
        def mount(self, *_args, **_kwargs):
            raise ModuleNotFoundError(
                "google.colab.drive は Colab 以外では使用できません"
            )
    class _DummyUserData(dict):
        def get(self, key, default=None):
            raise ModuleNotFoundError(
                "google.colab.userdata は Colab 以外では使用できません"
            )
    drive = _DummyDrive()
    userdata = _DummyUserData()
else:
    from google.colab import drive, userdata

# micromamba のパス
MICROMAMBA = Path('/usr/local/bin/micromamba')

def run(cmd, *, cwd=None, check=True, **kwargs):
    """コマンドを実行するヘルパー関数"""
    if isinstance(cmd, (list, tuple)):
        printable = ' '.join(str(c) for c in cmd)
    else:
        printable = cmd
    print(f"$ {printable}")
    return subprocess.run(cmd, cwd=cwd, check=check, **kwargs)

def ensure_micromamba():
    """micromamba をインストール"""
    if MICROMAMBA.exists():
        print('✅ micromamba は既にインストール済みです')
        return
    
    print('📦 micromamba をダウンロード中...')
    archive = Path('/tmp/micromamba.tar.bz2')
    run(['wget', '-qO', str(archive), 
         'https://micromamba.snakepit.net/api/micromamba/linux-64/latest'])
    run(['tar', '-xjf', str(archive), '-C', '/usr/local/bin', 
         '--strip-components=1', 'bin/micromamba'])
    print('✅ micromamba のインストールが完了しました')

def ensure_directories():
    """必要なディレクトリを作成"""
    drive_base_data_path = Path('/content/drive/MyDrive/sd_forge_data')
    drive_models_path = drive_base_data_path / 'models/Stable-diffusion'
    drive_controlnet_path = drive_base_data_path / 'models/ControlNet'
    drive_vae_path = drive_base_data_path / 'models/VAE'
    drive_lora_path = drive_base_data_path / 'models/Lora'
    
    for p in (drive_models_path, drive_controlnet_path, drive_vae_path, drive_lora_path):
        p.mkdir(parents=True, exist_ok=True)
    
    return drive_models_path, drive_controlnet_path, drive_vae_path, drive_lora_path

def download_civitai_model(token: str, drive_models_path: Path):
    """Civitai から WAI-NSFW-illustrious-SDXL モデルをダウンロード"""
    print("\n🔑 Civitai API トークンを使用してモデルをダウンロード...")
    
    def get_latest_model_version_id(token: str):
        print("🔍 WAI-NSFW-illustrious-SDXL の最新バージョンIDを取得中...")
        model_id = '827184'
        api_url = f"https://civitai.com/api/v1/models/{model_id}?token={token}"
        
        r = run(['curl', '-s', api_url], capture_output=True, text=True)
        if r.returncode != 0 or not r.stdout:
            print("⚠️ API呼び出しエラー、既知IDへフォールバックします")
            return None
        
        try:
            data = json.loads(r.stdout)
            mv = data.get('modelVersions') or []
            if mv:
                vid = str(mv[0]['id'])
                print(f"✅ 最新バージョンIDを取得: {vid}")
                return vid
        except json.JSONDecodeError:
            print("⚠️ APIレスポンスを解析できませんでした")
        return None
    
    def download_model(token: str, version_id: str):
        api = f"https://civitai.com/api/v1/model-versions/{version_id}?token={token}"
        r = run(['curl', '-s', api], capture_output=True, text=True)
        if r.returncode != 0 or not r.stdout:
            raise SystemExit('❌ モデル情報の取得に失敗しました')
        
        data = json.loads(r.stdout)
        files = data.get('files') or []
        chosen = next((f for f in files if f.get('type') == 'Model'), None)
        if not chosen:
            raise SystemExit('❌ モデルファイルが見つかりません')
        
        download_url = chosen['downloadUrl'] + f"?token={token}"
        output_filename = chosen['name']
        output_drive = drive_models_path / output_filename
        output_local = Path('/content/stable-diffusion-webui-forge/models/Stable-diffusion') / output_filename
        
        # 既にダウンロード済みかチェック
        if output_drive.exists() and output_drive.stat().st_size > 1_000_000:
            print(f"✅ {output_filename} は既にダウンロード済みです")
            output_local.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(output_drive, output_local)
            print(f"✅ Google Driveからローカルにコピーしました")
            return
        
        print(f"⬇️ {output_filename} をダウンロードします (数分かかる場合があります)...")
        aria_args = ['aria2c', '-c', '-x16', '-s16', '--check-certificate=false',
                     download_url, '-d', str(drive_models_path), '-o', output_filename]
        
        result = run(aria_args)
        if result.returncode == 0 and output_drive.exists():
            output_local.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(output_drive, output_local)
            print('✅ Civitai からモデルをダウンロードしました')
        else:
            raise SystemExit('❌ モデルのダウンロードに失敗しました')
    
    # 最新バージョンを取得してダウンロード
    vid = get_latest_model_version_id(token)
    if vid:
        download_model(token, vid)
        return
    
    # フォールバック: 既知のバージョンIDで試行
    fallback_versions = ['1410435', '1396035', '1378467', '1360950']
    for fallback in fallback_versions:
        try:
            print(f"⚠️ フォールバックバージョン {fallback} を試行中...")
            download_model(token, fallback)
            return
        except SystemExit:
            continue
    
    raise SystemExit('❌ すべてのモデルダウンロードに失敗しました')

def prepare_controlnet(drive_controlnet_path: Path):
    """ControlNet モデルをダウンロード"""
    print("\n🎮 ControlNet モデルを準備中...")
    controlnet_dir_local = Path('/content/stable-diffusion-webui-forge/models/ControlNet')
    controlnet_dir_local.mkdir(parents=True, exist_ok=True)
    
    models = [
        ('OpenPoseXL2.safetensors',
         'https://huggingface.co/thibaud/controlnet-openpose-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors'),
        ('diffusers_xl_canny_full.safetensors',
         'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
    ]
    
    for name, url in models:
        p_drive = drive_controlnet_path / name
        p_local = controlnet_dir_local / name
        
        # 既にダウンロード済みかチェック
        if p_drive.exists() and p_drive.stat().st_size > 1_000_000:
            print(f"✅ {name} は既にダウンロード済みです")
            shutil.copy(p_drive, p_local)
            continue
        
        print(f"⬇️ {name} をダウンロード中...")
        run(['aria2c', '-c', '-x8', '-s8', url, '-d', str(drive_controlnet_path), '-o', name])
        if p_drive.exists():
            shutil.copy(p_drive, p_local)
    
    print('✅ ControlNet モデル準備完了')

def write_config():
    """WebUI 設定ファイルを作成"""
    config = """{
  "samples_save": true,
  "samples_format": "png",
  "grid_save": true,
  "grid_format": "png",
  "enable_pnginfo": true,
  "save_selected_only": true,
  "jpeg_quality": 80,
  "export_for_4chan": true,
  "img_downscale_threshold": 4.0,
  "target_side_length": 4000,
  "img_max_size_mp": 200,
  "hashing_checkpoints_limit": 0,
  "sd_checkpoint_hash": ""
}"""
    cfg_path = Path('/content/stable-diffusion-webui-forge/config.json')
    cfg_path.parent.mkdir(parents=True, exist_ok=True)
    cfg_path.write_text(config)
    print('⚙️ SHA256計算をスキップする設定を追加しました')

def play_notification():
    """起動完了時に音を鳴らす"""
    if np is None:
        print('🔔 起動完了 (サウンド再生はスキップされました)')
        return
    
    sr = 22050
    t = np.linspace(0, 0.75, int(sr * 0.75), endpoint=False)
    tone = 0.3 * np.sin(2 * np.pi * 880 * t) * np.exp(-3 * t)
    display(Audio(data=tone, rate=sr, autoplay=True))

def start_gradio_live_monitor(url: str, proc: subprocess.Popen) -> None:
    """Gradio Live URL の疎通確認を行い、アクセス可能になったら通知"""
    url = (url or '').strip()
    if not url:
        print('⚠️ Gradio Live のURLを特定できませんでしたが通知を再生します')
        play_notification()
        return
    
    def _worker() -> None:
        print(f'⏳ Gradio Live URL の疎通確認を開始します: {url}')
        while True:
            if proc.poll() is not None:
                print('⚠️ WebUI プロセスが終了したため通知待機を中断します')
                return
            
            try:
                with urllib.request.urlopen(url, timeout=10) as response:
                    status = getattr(response, 'status', None)
                    if status is None:
                        status = response.getcode()
                    if status is None:
                        status = 200
                    if 200 <= status < 400:
                        print(f'✅ Gradio Live にアクセスできました (status={status})')
                        play_notification()
                        return
                    print(f'   ↪︎ 応答を受信しました (status={status}) - 再確認します')
            except urllib.error.HTTPError as exc:
                print(f'   ↪︎ アクセス待機中... (HTTP {exc.code})')
            except urllib.error.URLError as exc:
                print(f'   ↪︎ アクセス待機中... ({exc.reason})')
            except Exception as exc:
                print(f'   ↪︎ アクセス待機中... ({exc})')
            
            time.sleep(3)
    
    threading.Thread(target=_worker, name='gradio-live-monitor', daemon=True).start()

_active_processes = []

def launch_webui():
    """WebUI Forge を起動"""
    env = os.environ.copy()
    env['TORCH_COMMAND'] = 'pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121'
    
    cmd = [
        str(MICROMAMBA), 'run', '-n', 'py310', 'python', 'launch.py',
        '--share',  # Gradio Live で外部アクセスを有効化
        '--listen',
        '--xformers',  # メモリ最適化
        '--enable-insecure-extension-access',
        '--no-half-vae',  # VAE精度向上
        '--api',  # API有効化
        '--cors-allow-origins=*',  # CORS許可
        '--disable-console-progressbars',
        f'--lora-dir=/content/stable-diffusion-webui-forge/models/Lora'
    ]
    
    print('\n🚀 Stable Diffusion WebUI Forge を起動します...')
    print('📝 起動オプション: --share --xformers --no-half-vae')
    print('ℹ️  Python 3.10.13 環境で実行します（micromamba）')
    print('ℹ️  Forgeは自動でメモリ管理を行います')
    
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        env=env,
        cwd='/content/stable-diffusion-webui-forge'
    )
    _active_processes.append(proc)
    
    def monitor():
        notified = False
        stream = proc.stdout
        if stream is None:
            proc.wait()
            if proc.returncode not in (0, None):
                print(f"\n❌ WebUI が終了しました (returncode={proc.returncode})")
            return
        
        try:
            for line in iter(stream.readline, ''):
                if not line:
                    break
                sys.stdout.write(line)
                
                # Gradio Live URL を検出
                if not notified and 'Running on public URL:' in line:
                    public_url = line.split('Running on public URL:', 1)[1].strip()
                    start_gradio_live_monitor(public_url, proc)
                    notified = True
        finally:
            proc.wait()
            if proc.returncode not in (0, None):
                print(f"\n❌ WebUI が終了しました (returncode={proc.returncode})")
    
    try:
        monitor()
    except KeyboardInterrupt:
        print("\n⏹️ WebUI 停止要求を受信しました。プロセスを終了します...")
        if proc.poll() is None:
            proc.terminate()
            try:
                proc.wait(timeout=10)
            except subprocess.TimeoutExpired:
                proc.kill()
        return
    finally:
        if proc.stdout is not None:
            proc.stdout.close()
        if proc in _active_processes:
            _active_processes.remove(proc)

# ========== メインの実行フロー ==========

print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print('🎨 Stable Diffusion WebUI Forge セットアップ')
print('   (Python 3.10 + micromamba版)')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')

# Python バージョンを表示
print(f'\n🐍 Colab Python バージョン: {sys.version}')

# 1. apt パッケージのインストール
print('\n📦 apt パッケージをインストール中...')
run(['apt-get', 'update', '-qq'])
run(['apt-get', 'install', '-y', '-qq', 'aria2', 'wget', 'curl', 'libgl1', 'libglib2.0-0'])

# 2. micromamba のセットアップ
print('\n📦 micromamba をセットアップ中...')
ensure_micromamba()

print('\n🐍 Python 3.10.13 環境を作成中...')
run([str(MICROMAMBA), 'create', '-y', '-n', 'py310', '-c', 'conda-forge', 
     'python=3.10.13', 'pip', 'git', 'aria2', 'curl'])

print('\n🧪 Python 3.10 環境のバージョン確認：')
run([str(MICROMAMBA), 'run', '-n', 'py310', 'python', '-V'])

# 3. Google Drive のマウント
drive_mount_point = Path('/content/drive')
if drive_mount_point.exists() and (drive_mount_point / 'MyDrive').exists():
    print('\n✅ Google Drive は既にマウント済みです')
else:
    try:
        print('\n📁 Google Drive をマウントします（初回のみ許可が必要です）...')
        drive.mount('/content/drive')
        print('✅ Google Drive をマウントしました')
    except Exception as e:
        print(f'⚠️ Drive マウント失敗: {e}')
        raise

# 4. 必要なディレクトリを作成
drive_models_path, drive_controlnet_path, drive_vae_path, drive_lora_path = ensure_directories()

# 5. Forge版リポジトリのクローン
webui_path = Path('/content/stable-diffusion-webui-forge')
if webui_path.exists():
    print(f'\n🧹 既存のWebUIを削除: {webui_path}')
    shutil.rmtree(webui_path)

print('\n⬇️ Stable Diffusion WebUI Forge をクローン中...')
os.chdir('/content')
run(['git', 'clone', 'https://github.com/lllyasviel/stable-diffusion-webui-forge.git'])
print('✅ Forge版のクローンが完了しました')

# 6. PyTorch と追加ライブラリをインストール (py310 環境内)
print('\n🔥 PyTorch (cu121) を py310 環境へインストール...')
run([str(MICROMAMBA), 'run', '-n', 'py310', 'python', '-m', 'pip', 'install', '--upgrade', 'pip'])
run([str(MICROMAMBA), 'run', '-n', 'py310', 'pip', 'install', 
     '--index-url', 'https://download.pytorch.org/whl/cu121', 
     'torch', 'torchvision', 'torchaudio'])

print('\n📚 追加ライブラリ（py310 環境）をインストール...')
run([str(MICROMAMBA), 'run', '-n', 'py310', 'pip', 'install', 
     'open_clip_torch', 'transformers', 'timm', 'requests'])

# 7. 拡張機能の確認（ControlNet は Forge に統合済み）
print('\n🔌 拡張機能の確認...')
print('✅ Forge版にはControlNetが統合されています')

# 8. CIVITAI_API_TOKEN の取得
print('\n🔑 CIVITAI_API_TOKEN を検出中...')
try:
    civitai_token = (userdata.get('CIVITAI_API_TOKEN') or '').strip()
    if not civitai_token:
        raise ValueError('CIVITAI_API_TOKEN が未設定')
    os.environ['CIVITAI_API_TOKEN'] = civitai_token
    print('✅ CIVITAI_API_TOKEN を検出')
except Exception as e:
    print('❌ Civitai APIトークンが見つかりません')
    print('   → サイドバーの 🔑 Secrets で `CIVITAI_API_TOKEN` を設定して再実行してください。')
    print('   → トークンは https://civitai.com/user/account で取得できます。')
    raise

# 9. WAI-NSFW-illustrious-SDXL モデルのダウンロード
os.chdir('/content/stable-diffusion-webui-forge')
download_civitai_model(civitai_token, drive_models_path)

# 10. ControlNet モデルのダウンロード
prepare_controlnet(drive_controlnet_path)

# 11. 設定ファイルの作成
write_config()

# 12. Google Drive のシンボリックリンクを作成（永続化）
print('\n🔗 Google Drive とのシンボリックリンクを作成中...')
os.chdir('/content/stable-diffusion-webui-forge')

# models ディレクトリのシンボリックリンク
for dir_name in ['Stable-diffusion', 'Lora', 'VAE', 'ControlNet']:
    local_path = Path(f'/content/stable-diffusion-webui-forge/models/{dir_name}')
    drive_path = Path(f'/content/drive/MyDrive/sd_forge_data/models/{dir_name}')
    
    if local_path.exists() and not local_path.is_symlink():
        shutil.rmtree(local_path)
    elif local_path.is_symlink():
        local_path.unlink()
    
    local_path.symlink_to(drive_path)
    print(f'✅ {dir_name} → Google Drive')

# 13. matplotlib のバックエンド設定を削除（環境互換性のため）
os.environ.pop('MPLBACKEND', None)

# 14. insightface をインストール (py310 環境内)
print('\n🔎 insightface (追加) をインストール...')
run([str(MICROMAMBA), 'run', '-n', 'py310', 'pip', 'install', 'insightface'])

# 15. WebUI Forge を起動
print('\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print('🎉 セットアップ完了！WebUI Forge を起動します')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')

launch_webui()